In [3]:
import os
import numpy as np
from nipype import Function, Workflow, config, logging
import nipype.interfaces.utility as niu
import nipype.pipeline.engine as pe
from nipype.interfaces import io

from mribrew.utils import colours
from mribrew.data_io import read_dwi_data
from mribrew.mapmri_funcs import correct_neg_data, fit_mapmri_model, metrics_to_nifti, correct_metric_nifti

# ---------------------- Set up directory structures and constant variables ----------------------
cwd = os.getcwd()
data_dir = os.path.join(cwd, 'data')
raw_dir = os.path.join(data_dir, 'raw')
proc_dir = os.path.join(data_dir, 'proc')
wf_dir = os.path.join(cwd, 'wf')
res_dir = os.path.join(data_dir, 'res')
log_dir = os.path.join(wf_dir, 'log')

subject_list = next(os.walk(proc_dir))[1]  # processed subjects

# Computational variables
use_subset_data = True
processing_type = 'MultiProc' # or 'Linear'
n_cpus = 12

# MAPMRI variables
big_delta, small_delta = 0.0353, 0.0150
mapmri_params = dict(radial_order=4,
                     laplacian_regularization=True,
                     laplacian_weighting=0.2,
                     positivity_constraint=False,
                     global_constraints=False,
                     pos_grid=15,
                     pos_radius='adaptive',
                     anisotropic_scaling=True,
                     eigenvalue_threshold=1e-04,
                     bval_threshold=np.inf,
                     dti_scale_estimation=True,
                     static_diffusivity=0.7e-3,
                     cvxpy_solver=None)

# Set up logging
os.makedirs(log_dir, exist_ok=True)
config.update_config({'logging': {'log_directory': log_dir,'log_to_file': True}})
logging.update_logging(config)

print(f"\n{colours.UBOLD}{colours.CYELLOW}Starting the MAPMRI pipeline...{colours.CEND}")
print(f"Using the following constants:\n"
      f"MAPMRI Parameters: {mapmri_params}\n"
      f"Small delta: {small_delta}\n"
      f"Big delta: {big_delta}\n"
      f"Using subset data: {use_subset_data}\n"
      f"Number of CPUs: {n_cpus}\n"
      f"Processing type: {processing_type}\n")

# ---------------------- INPUT SOURCE NODES ----------------------
print(colours.CGREEN + "Creating Source Nodes." + colours.CEND)

# Set up input files
info = dict(dwi_eddy_file=[['subject_id', 'dwi', 'eddy_corrected.nii.gz']],
            bvec_file=[['subject_id', 'dwi', 'gradChecked.bvec']],
            bval_file=[['subject_id', 'dwi', 'gradChecked.bval']],
            dwi_mask_file=[['subject_id', 'dwi', 'brain_dwi_mask.nii.gz']])

# Set up infosource node
infosource = pe.Node(niu.IdentityInterface(fields=['subject_id']), name='infosource')
infosource.iterables = [('subject_id', subject_list)]
infosource.inputs.big_delta = big_delta
infosource.inputs.small_delta = small_delta
infosource.inputs.use_subset_data = use_subset_data

# Set up datasource node
datasource = pe.Node(io.DataGrabber(infields=['subject_id'], outfields=list(info.keys())),
                                    name='datasource')
datasource.inputs.base_directory = proc_dir
datasource.inputs.template = "%s/%s/%s"
datasource.inputs.template_args = info
datasource.inputs.sort_filelist = True

# ---------------------- PROCESSING NODES ----------------------
print(colours.CGREEN + "Creating Processing Nodes." + colours.CEND)

# Set up a node for loading in DWI data (and apply mask) and gradient table
read_data = pe.Node(Function(input_names=['data_file', 'mask_file',
                                          'bvec_file', 'bval_file',
                                          'big_delta', 'small_delta',
                                          'use_subset_data'],
                             output_names=['data', 'affine', 'gtab'],
                             function=read_dwi_data),
                    name='read_data')
read_data.inputs.big_delta = big_delta
read_data.inputs.small_delta = small_delta
read_data.inputs.use_subset_data = use_subset_data

# Set up a node for correcting negative values to average volume value per timepoint
data_correction = pe.Node(Function(input_names=['data'],
                                   output_names=['data_corrected'],
                                   function=correct_neg_data),
                          name='data_correction')

# Set up a node for fitting data to the MAPMRI model
fit_mapmri = pe.Node(Function(input_names=['data', 'gtab', 'mapmri_params'],
                              output_names=['MSD', 'QIV', 'RTOP', 'RTAP', 'RTPP'],
                              function=fit_mapmri_model),
                     name='fit_mapmri')
fit_mapmri.inputs.mapmri_params = mapmri_params

# ---------------------- OUTPUT NODES ----------------------
print(colours.CGREEN + "Creating Output Nodes." + colours.CEND)

# Set up a node for saving metrics as NIfTI
metrics_to_nii = pe.Node(Function(input_names=['affine', 'MSD', 'QIV', 'RTOP', 'RTAP', 
                                               'RTPP', 'out_file_prefix', 'res_dir'],
                              output_names=['MSD_file', 'QIV_file', 'RTOP_file', 
                                            'RTAP_file', 'RTPP_file'],
                              function=metrics_to_nifti),
                     name='metrics_to_nii')
metrics_to_nii.inputs.res_dir = os.path.join(res_dir, 'mapmri')

# Set up a node for correcting RTOP metric and saving as NIfTI 
rtop_corrected_to_nii = pe.Node(Function(input_names=['metric_path', 'threshold', 
                                                      'correct_neg', 'replace_with'],
                              output_names=['out_file'],
                              function=correct_metric_nifti),
                     name='rtop_corrected_to_nii')
rtop_corrected_to_nii.inputs.threshold = 2000000
rtop_corrected_to_nii.inputs.correct_neg = True
rtop_corrected_to_nii.inputs.replace_with = 0

# ---------------------- CREATE WORKFLOW AND CONNECT NODES ----------------------
print(colours.CGREEN + 'Connecting Nodes.\n' + colours.CEND)

workflow = Workflow(name='mapmri_wf', base_dir=f"{wf_dir}")
workflow.connect([
    (infosource, datasource, [('subject_id', 'subject_id')]),
    (datasource, read_data, [('dwi_eddy_file', 'data_file'),
                             ('dwi_mask_file', 'mask_file'),
                             ('bvec_file', 'bvec_file'),
                             ('bval_file', 'bval_file')]),
    (read_data, data_correction, [('data', 'data')]),
    (data_correction, fit_mapmri, [('data_corrected', 'data')]),
    (read_data, fit_mapmri, [('gtab', 'gtab')]),
    (fit_mapmri, metrics_to_nii, [('MSD', 'MSD'),
                               ('QIV', 'QIV'),
                               ('RTOP', 'RTOP'),
                               ('RTAP', 'RTAP'),
                               ('RTPP', 'RTPP')]),
    (read_data, metrics_to_nii, [('affine', 'affine')]),
    (infosource, metrics_to_nii, [('subject_id', 'out_file_prefix')]),
    (metrics_to_nii, rtop_corrected_to_nii, [('RTOP_file', 'metric_path')])
])

if __name__ == '__main__':
    workflow.write_graph(graph2use='orig')
    workflow.run(plugin=processing_type, plugin_args={'n_procs' : n_cpus})



Starting the MAPMRI pipeline...
Using the following constants:
MAPMRI Parameters: {'radial_order': 4, 'laplacian_regularization': True, 'laplacian_weighting': 0.2, 'positivity_constraint': False, 'global_constraints': False, 'pos_grid': 15, 'pos_radius': 'adaptive', 'anisotropic_scaling': True, 'eigenvalue_threshold': 0.0001, 'bval_threshold': inf, 'dti_scale_estimation': True, 'static_diffusivity': 0.0007, 'cvxpy_solver': None}
Small delta: 0.015
Big delta: 0.0353
Using subset data: True
Number of CPUs: 12
Processing type: MultiProc

Creating Source Nodes.
Creating Processing Nodes.
Creating Output Nodes.
Connecting Nodes.

230818-14:54:01,366 nipype.workflow INFO:
	 Generated workflow graph: /Users/tanijarv/Documents/GitHub/mribrew/wf/mapmri_wf/graph.png (graph2use=orig, simple_form=True).
230818-14:54:01,390 nipype.workflow INFO:
	 Workflow mapmri_wf settings: ['check', 'execution', 'logging', 'monitoring']
230818-14:54:01,399 nipype.workflow INFO:
	 Running in parallel.
230818-14:

  0%|          | 0/4400 [00:00<?, ?it/s]/opt/anaconda3/envs/mri/lib/python3.11/site-packages/dipy/reconst/mapmri.py:488: RuntimeWarning: invalid value encountered in divide
  coef = coef / sum(coef * self.Bm)
/opt/anaconda3/envs/mri/lib/python3.11/site-packages/dipy/reconst/mapmri.py:488: RuntimeWarning: invalid value encountered in divide
  coef = coef / sum(coef * self.Bm)
  5%|▌         | 225/4400 [00:00<00:13, 317.93it/s]

230818-14:54:09,414 nipype.workflow INFO:
	 [MultiProc] Running 2 tasks, and 0 jobs ready. Free memory (GB): 14.00/14.40, Free processors: 10/12.
                     Currently running:
                       * mapmri_wf.fit_mapmri
                       * mapmri_wf.fit_mapmri


100%|██████████| 4400/4400 [00:14<00:00, 311.57it/s]


230818-14:54:23,493 nipype.workflow INFO:
	 [Node] Finished "fit_mapmri", elapsed time 15.442741s.
230818-14:54:23,537 nipype.workflow INFO:
	 [Node] Finished "fit_mapmri", elapsed time 15.471365s.
230818-14:54:25,448 nipype.workflow INFO:
	 [Job 6] Completed (mapmri_wf.fit_mapmri).
230818-14:54:25,450 nipype.workflow INFO:
	 [Job 7] Completed (mapmri_wf.fit_mapmri).
230818-14:54:25,453 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 2 jobs ready. Free memory (GB): 14.40/14.40, Free processors: 12/12.
230818-14:54:25,525 nipype.workflow INFO:
	 [Job 8] Cached (mapmri_wf.metrics_to_nii).
230818-14:54:25,554 nipype.workflow INFO:
	 [Job 9] Cached (mapmri_wf.metrics_to_nii).
230818-14:54:27,491 nipype.workflow INFO:
	 [Job 10] Cached (mapmri_wf.rtop_corrected_to_nii).
230818-14:54:27,494 nipype.workflow INFO:
	 [Job 11] Cached (mapmri_wf.rtop_corrected_to_nii).
